<a href="https://colab.research.google.com/github/jmo2171/ml-climate/blob/main/src/wildfire_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

### imports

In [1]:
import zipfile
import sqlite3
import pandas as pd
from tqdm import tqdm
import numpy as np
import glob
import ast

In [2]:
import datetime

def julian_to_datetime(julian_date):
    # Julian Day 0 = 4713-11-24 BCE (proleptic Gregorian calendar)
    # Astronomical JD 2440587.5 = Unix time 0 (1970-01-01T00:00:00Z)
    unix_time_seconds = (julian_date - 2440587.5) * 86400.0
    return datetime.datetime.utcfromtimestamp(unix_time_seconds)

### google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### google earth engine

In [4]:
import ee

In [5]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='cvproject-421022')

# load 1.88 mil wildfire data

In [6]:
with zipfile.ZipFile('/content/drive/MyDrive/Spring 2025/ML and Climate/wildfire_project/fod.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/fod')


In [31]:
con = sqlite3.connect('/content/fod/FPA_FOD_20170508.sqlite')
cur = con.cursor()

# for row in cur.execute('SELECT COUNT(*) FROM Fires F WHERE F.STATE="CA"'):
#   print(row)


# for row in cur.execute('SELECT FOD_ID, FIRE_NAME, FIRE_SIZE, FIRE_SIZE_CLASS, FIRE_YEAR, DISCOVERY_DATE, STAT_CAUSE_DESCR, STATE, LATITUDE, LONGITUDE FROM Fires F WHERE F.STATE="CA" LIMIT 2'):
#   print(row)
# con.close()

ca_fires = pd.read_sql('SELECT FOD_ID, FIRE_NAME, FIRE_SIZE, FIRE_SIZE_CLASS, FIRE_YEAR, DISCOVERY_DATE, STAT_CAUSE_DESCR, STATE, LATITUDE, LONGITUDE FROM Fires F WHERE F.STATE="CA"', con)
# all_fires = pd.read_sql('SELECT FOD_ID, FIRE_NAME, FIRE_SIZE, FIRE_SIZE_CLASS, FIRE_YEAR, DISCOVERY_DATE, STAT_CAUSE_DESCR, STATE, LATITUDE, LONGITUDE FROM Fires F', con)


In [ ]:
test_fires = pd.read_sql('SELECT FOD_ID, FIRE_NAME, FIRE_SIZE, FIRE_SIZE_CLASS, FIRE_YEAR, DISCOVERY_DATE, STAT_CAUSE_DESCR, STATE, LATITUDE, LONGITUDE FROM Fires F LIMIT 15', con)
test_fires

In [8]:
# ca_fires[ca_fires['FIRE_SIZE_CLASS']=='E']
ca_fires['FIRE_SIZE_CLASS'].unique()

array(['A', 'B', 'G', 'C', 'F', 'D', 'E'], dtype=object)

In [33]:
ca_fires["date"] = ca_fires["DISCOVERY_DATE"].apply(julian_to_datetime)

In [34]:
print(ca_fires['FOD_ID'].nunique())
print(ca_fires['date'].nunique())
ca_fires[(ca_fires['LATITUDE']==37.375) & (ca_fires['LONGITUDE']==-120.62305555) & (ca_fires['date']=='2007-09-28 0:00:00')]

print(ca_fires[ca_fires['date']>='2000-02-18'].shape[0])
ca_2000_fires = ca_fires[ca_fires['date']>='2000-02-18']

189550
8324
121328


# augment with earth engine data

## single point test

In [ ]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
# maximumTemperature = dataset.select('tmmx')



row = ca_fires.iloc[0]
print(row)
print(julian_to_datetime(row['DISCOVERY_DATE']))
idx = 1
point = ee.Geometry.Point([row['LONGITUDE'], row['LATITUDE']])
feature = ee.Feature(point, {
            'id': str(idx),
            'date': julian_to_datetime(row['DISCOVERY_DATE'])
        })
date = ee.Date(feature.get('date'))

image = gridmet.filterDate(date, date.advance(1, 'day')).first()
max_temp = image.select('tmmx')
# print(image)
max_temp_value = max_temp.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=feature.geometry(),
        scale=1000
    ).get('tmmx').getInfo()
print(max_temp_value)
# Print the elevation near Lyon, France.
# lst_urban_point = dataset.mean().sample(u_poi, scale).first().get('tmmx').getInfo()
# print(lst_urban_point - 273.15)
# print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')


FOD_ID                          1
FIRE_NAME                FOUNTAIN
FIRE_SIZE                     0.1
FIRE_SIZE_CLASS                 A
FIRE_YEAR                    2005
DISCOVERY_DATE          2453403.5
STAT_CAUSE_DESCR    Miscellaneous
STATE                          CA
LATITUDE                40.036944
LONGITUDE             -121.005833
Name: 0, dtype: object
2005-02-02 00:00:00
280.87799072265625


In [ ]:
pop = ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Density')

row = ca_fires.iloc[0]
print(row)
print(julian_to_datetime(row['DISCOVERY_DATE']))
idx = 1
point = ee.Geometry.Point([row['LONGITUDE'], row['LATITUDE']])
feature = ee.Feature(point, {
            'id': str(idx),
            'date': julian_to_datetime(row['DISCOVERY_DATE']),
            "year": 5 * round(julian_to_datetime(row['DISCOVERY_DATE']).year/5)
        })
year = feature.get('year')
date = ee.Date.fromYMD(year, 1, 1)

image = pop.filterDate(date, date.advance(1, 'year')).first()
# print(image)


population_density = image.select('population_density')

# Sample at the feature's location
population_density_value = population_density.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=feature.geometry(),
    scale=1000).get('population_density').getInfo()

print(population_density_value)


FOD_ID                                1
FIRE_NAME                      FOUNTAIN
FIRE_SIZE                           0.1
FIRE_SIZE_CLASS                       A
FIRE_YEAR                          2005
DISCOVERY_DATE                2453403.5
STAT_CAUSE_DESCR          Miscellaneous
STATE                                CA
LATITUDE                      40.036944
LONGITUDE                   -121.005833
date                2005-02-02 00:00:00
Name: 0, dtype: object
2005-02-02 00:00:00
4.473303318023682


In [ ]:
veg = ee.ImageCollection('NASA/VIIRS/002/VNP13A1')


row = ca_fires.iloc[0]
print(row)
print(julian_to_datetime(row['DISCOVERY_DATE']))
idx = 1
point = ee.Geometry.Point([row['LONGITUDE'], row['LATITUDE']])
feature = ee.Feature(point, {
            'id': str(idx),
            'date': julian_to_datetime(row['DISCOVERY_DATE']),
            "year": 5 * round(julian_to_datetime(row['DISCOVERY_DATE']).year/5)
        })

date = ee.Date(feature.get('date'))
image = veg.filterDate(date, date.advance(1, 'month')).first()# print(image)
vegetation = image.select('EVI')

population_density = image.select('population_density')
# Sample at the feature's location
vegetation_value = vegetation.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=feature.geometry(),
    scale=10000
).get('EVI').getInfo()

print(vegetation_value)

FOD_ID                                1
FIRE_NAME                      FOUNTAIN
FIRE_SIZE                           0.1
FIRE_SIZE_CLASS                       A
FIRE_YEAR                          2005
DISCOVERY_DATE                2453403.5
STAT_CAUSE_DESCR          Miscellaneous
STATE                                CA
LATITUDE                      40.036944
LONGITUDE                   -121.005833
date                2005-02-02 00:00:00
Name: 0, dtype: object
2005-02-02 00:00:00


EEException: Image.select: Parameter 'input' is required and may not be null.

In [ ]:
veg = ee.ImageCollection('NASA/VIIRS/002/VNP13A1')


row = ca_fires.iloc[0]
print(row)
print(julian_to_datetime(row['DISCOVERY_DATE']))
idx = 1
point = ee.Geometry.Point([row['LONGITUDE'], row['LATITUDE']])
feature = ee.Feature(point, {
            'id': str(idx),
            'date': julian_to_datetime(row['DISCOVERY_DATE']),
            "year": 5 * round(julian_to_datetime(row['DISCOVERY_DATE']).year/5)
        })

date = ee.Date(feature.get('date'))
image = veg.filterDate(date, date.advance(1, 'month')).first()# print(image)
vegetation = image.select('EVI')

population_density = image.select('population_density')
# Sample at the feature's location
vegetation_value = vegetation.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=feature.geometry(),
    scale=10000
).get('EVI').getInfo()

print(vegetation_value)


def sample_drought(feature):

    date = ee.Date(feature.get('date'))
    image = veg.filterDate(date, date.advance(1, 'week')).first()

    drought = image.select('pdsi')

    # Sample at the feature's location
    drought_value = drought.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=feature.geometry(),
        scale=10000
    ).get('pdsi')


    return feature.set({
        'drought': drought_value,
    })

## batch processing

In [11]:
tqdm.pandas()

def df_to_fc(df):
    features = []
    for idx, row in tqdm(df.iterrows()):
        point = ee.Geometry.Point([row['LONGITUDE'], row['LATITUDE']])
        feature = ee.Feature(point, {
            'id': row['FOD_ID'],
            'date': julian_to_datetime(row['DISCOVERY_DATE']),
            "year": max(5 * round(julian_to_datetime(row['DISCOVERY_DATE']).year/5), 2000)
        })
        features.append(feature)
    return ee.FeatureCollection(features)


### gridmet weather data


In [14]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')

def sample_weather(feature):
    date = ee.Date(feature.get('date'))
    image = gridmet.filterDate(date, date.advance(1, 'day')).first()

    max_temp = image.select('tmmx')
    min_temp = image.select('tmmn')
    humidity = image.select('sph')
    wind_speed = image.select('vs')
    precipitation = image.select('pr')
    energy_release = image.select('erc')
    dead_fuel_100 = image.select('fm100')
    dead_fuel_1000 = image.select('fm1000')

    # Sample at the feature's location
    max_temp_value = max_temp.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('tmmx')

    min_temp_value = min_temp.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('tmmn')

    humidity_value = humidity.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('sph')

    wind_speed_value = wind_speed.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('vs')

    precipitation_value = precipitation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('pr')

    energy_release_value = energy_release.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('erc')

    dead_fuel_100_value = dead_fuel_100.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('fm100')

    dead_fuel_1000_value = dead_fuel_1000.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('fm1000')



    return feature.set({
        'max_temp_K': max_temp_value,
        'min_temp_K': min_temp_value,
        'humidity': humidity_value,
        'wind_speed' : wind_speed_value,
        'precipitation' : precipitation_value,
        'energy_release' : energy_release_value,
        'dead_fuel_100' : dead_fuel_100_value,
        'dead_fuel_1000' : dead_fuel_1000_value
    })



In [16]:
chunk_size = 20000
chunks = np.array_split(ca_fires, np.ceil(len(ca_fires) / chunk_size))



for idx, chunk in enumerate(chunks):
    print(f'Processing chunk {idx+1}/{len(chunks)}...')

    # Convert this chunk into a FeatureCollection
    fc_chunk = df_to_fc(chunk)

    # Sample weather
    sampled_chunk = fc_chunk.map(sample_weather)

    # Export this chunk
    task = ee.batch.Export.table.toDrive(
        collection=sampled_chunk,
        description=f'export_weather_points_batch_{idx}',
        fileFormat='CSV',
        folder='earthengine_try3',
        fileNamePrefix=f'weather_points_batch_{idx}'
    )
    task.start()

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processing chunk 1/10...


18955it [00:02, 6877.56it/s]


Processing chunk 2/10...


18955it [00:02, 9311.33it/s]


...

18955it [00:03, 5026.13it/s]


Processing chunk 9/10...


18955it [00:02, 9177.66it/s]


Processing chunk 10/10...


18955it [00:02, 9186.30it/s]


### population gpwv4 data

In [17]:
pop = ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Density')

def sample_population(feature):

    year = feature.get('year')
    date = ee.Date.fromYMD(year, 1, 1)
    image = pop.filterDate(date, date.advance(1, 'year')).first()

    population_density = image.select('population_density')

    # Sample at the feature's location
    population_density_value = population_density.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('population_density')


    return feature.set({
        'population_density': population_density_value,
    })



In [20]:
chunk_size = 20000
chunks = np.array_split(ca_2000_fires, np.ceil(len(ca_2000_fires) / chunk_size))

for idx, chunk in enumerate(chunks):
    print(f'Processing chunk {idx+1}/{len(chunks)}...')

    # Convert this chunk into a FeatureCollection
    fc_chunk = df_to_fc(chunk)

    # Sample weather
    sampled_chunk = fc_chunk.map(sample_population)

    # Export this chunk
    task = ee.batch.Export.table.toDrive(
        collection=sampled_chunk,
        description=f'export_population_points_batch_{idx}',
        fileFormat='CSV',
        folder='earthengine_try3',
        fileNamePrefix=f'population_points_batch_{idx}'
    )
    task.start()

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processing chunk 1/7...


17333it [00:02, 6651.33it/s]


Processing chunk 2/7...


17333it [00:02, 5865.21it/s]


...

17332it [00:02, 6968.06it/s]


Processing chunk 6/7...


17332it [00:01, 9108.52it/s]


Processing chunk 7/7...


17332it [00:02, 5970.87it/s]


### vegetation modi3

In [21]:
veg = ee.ImageCollection('MODIS/061/MOD13A2')

def sample_vegetation(feature):

    date = ee.Date(feature.get('date'))
    image = veg.filterDate(date, date.advance(1, 'month')).first()

    vegetation = image.select('EVI')

    # Sample at the feature's location
    vegetation_value = vegetation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('EVI')


    return feature.set({
        'vegetation': vegetation_value,
    })



In [22]:
chunk_size = 20000
chunks = np.array_split(ca_2000_fires, np.ceil(len(ca_2000_fires) / chunk_size))

for idx, chunk in enumerate(chunks):
    print(f'Processing chunk {idx+1}/{len(chunks)}...')

    # Convert this chunk into a FeatureCollection
    fc_chunk = df_to_fc(chunk)

    # Sample weather
    sampled_chunk = fc_chunk.map(sample_vegetation)

    # Export this chunk
    task = ee.batch.Export.table.toDrive(
        collection=sampled_chunk,
        description=f'export_vegetation_points_batch_{idx}',
        fileFormat='CSV',
        folder='earthengine_try3',
        fileNamePrefix=f'vegetation_points_batch_{idx}'
    )
    task.start()

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processing chunk 1/7...


17333it [00:01, 9191.79it/s]


Processing chunk 2/7...


17333it [00:03, 5204.81it/s]


...

17332it [00:02, 7682.86it/s]


Processing chunk 6/7...


17332it [00:02, 6036.82it/s]


Processing chunk 7/7...


17332it [00:01, 9065.61it/s]


### drought gridmet drought

In [23]:
drought = ee.ImageCollection('GRIDMET/DROUGHT')

def sample_drought(feature):

    date = ee.Date(feature.get('date'))
    image = drought.filterDate(date, date.advance(1, 'week')).first()

    drought_index = image.select('pdsi')

    # Sample at the feature's location
    drought_index_value = drought_index.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('pdsi')


    return feature.set({
        'drought_index': drought_index_value,
    })


In [24]:
chunk_size = 20000
chunks = np.array_split(ca_2000_fires, np.ceil(len(ca_2000_fires) / chunk_size))

for idx, chunk in enumerate(chunks):
    print(f'Processing chunk {idx+1}/{len(chunks)}...')

    # Convert this chunk into a FeatureCollection
    fc_chunk = df_to_fc(chunk)

    # Sample weather
    sampled_chunk = fc_chunk.map(sample_drought)

    # Export this chunk
    task = ee.batch.Export.table.toDrive(
        collection=sampled_chunk,
        description=f'export_drought_points_batch_{idx}',
        fileFormat='CSV',
        folder='earthengine_try3',
        fileNamePrefix=f'drought_points_batch_{idx}'
    )
    task.start()

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processing chunk 1/7...


17333it [00:02, 6575.33it/s]


Processing chunk 2/7...


17333it [00:01, 9199.20it/s]


...

17332it [00:02, 6436.67it/s]


Processing chunk 6/7...


17332it [00:02, 7609.94it/s]


Processing chunk 7/7...


17332it [00:02, 7670.36it/s]


# merge data

In [35]:
batch_files = glob.glob('/content/drive/MyDrive/earthengine_try3/weather_points_batch_*.csv')

exported_df = pd.concat((pd.read_csv(f) for f in batch_files), ignore_index=True)
exported_df.rename(columns={'id': 'FOD_ID'}, inplace=True)
exported_df.drop_duplicates(subset=["FOD_ID"], inplace=True)

# Convert from Kelvin to Celsius
exported_df['max_temp_C'] = exported_df['max_temp_K'] - 273.15
exported_df['min_temp_C'] = exported_df['min_temp_K'] - 273.15

print(ca_fires['FOD_ID'].nunique())

print(exported_df.columns.tolist())



189550
['system:index', 'date', 'dead_fuel_100', 'dead_fuel_1000', 'energy_release', 'humidity', 'FOD_ID', 'max_temp_K', 'min_temp_K', 'precipitation', 'wind_speed', 'year', '.geo', 'max_temp_C', 'min_temp_C']


In [36]:

merged_df = ca_2000_fires.merge(exported_df[['FOD_ID', 'dead_fuel_100', 'dead_fuel_1000', 'energy_release', 'humidity', 'max_temp_C', 'min_temp_C', 'precipitation', 'wind_speed']],
                           on='FOD_ID', how='left')




In [37]:
batch_files = glob.glob('/content/drive/MyDrive/earthengine_try3/population_points_batch_*.csv')

exported_df = pd.concat((pd.read_csv(f) for f in batch_files), ignore_index=True)
exported_df.rename(columns={'id': 'FOD_ID'}, inplace=True)
exported_df.drop_duplicates(subset=["FOD_ID"], inplace=True)


In [38]:
merged_df = merged_df.merge(exported_df[['FOD_ID', 'population_density']],
                           on='FOD_ID', how='left')

In [39]:
batch_files = glob.glob('/content/drive/MyDrive/earthengine_try3/vegetation_points_batch_*.csv')

exported_df = pd.concat((pd.read_csv(f) for f in batch_files), ignore_index=True)
exported_df.rename(columns={'id': 'FOD_ID'}, inplace=True)
exported_df.drop_duplicates(subset=["FOD_ID"], inplace=True)


In [40]:
merged_df = merged_df.merge(exported_df[['FOD_ID', 'vegetation']],
                           on='FOD_ID', how='left')

In [41]:
batch_files = glob.glob('/content/drive/MyDrive/earthengine_try3/drought_points_batch_*.csv')

exported_df = pd.concat((pd.read_csv(f) for f in batch_files), ignore_index=True)
exported_df.rename(columns={'id': 'FOD_ID'}, inplace=True)
exported_df.drop_duplicates(subset=["FOD_ID"], inplace=True)


In [42]:
merged_df = merged_df.merge(exported_df[['FOD_ID', 'drought_index']],
                           on='FOD_ID', how='left')

In [43]:
print(merged_df.columns.tolist())


['FOD_ID', 'FIRE_NAME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'FIRE_YEAR', 'DISCOVERY_DATE', 'STAT_CAUSE_DESCR', 'STATE', 'LATITUDE', 'LONGITUDE', 'date', 'dead_fuel_100', 'dead_fuel_1000', 'energy_release', 'humidity', 'max_temp_C', 'min_temp_C', 'precipitation', 'wind_speed', 'population_density', 'vegetation', 'drought_index']


In [44]:
merged_df

,FOD_ID,FIRE_NAME,FIRE_SIZE,FIRE_SIZE_CLASS,FIRE_YEAR,DISCOVERY_DATE,STAT_CAUSE_DESCR,STATE,LATITUDE,LONGITUDE,...,dead_fuel_1000,energy_release,humidity,max_temp_C,min_temp_C,precipitation,wind_speed,population_density,vegetation,drought_index
0,1,FOUNTAIN,0.10,A,2005,2453403.5,Miscellaneous,CA,40.036944,-121.005833,...,21.963203,22.056320,0.002912,7.498254,-2.186072,0.0,6.620363,1.478607,2929.0,-0.3500
1,2,PIGEON,0.25,A,2004,2453137.5,Lightning,CA,38.933056,-120.404444,...,12.930136,47.595642,0.003536,15.886743,0.414148,0.0,3.208708,0.004660,3590.0,-2.2075
2,3,SLACK,0.10,A,2004,2453156.5,Debris Burning,CA,38.984167,-120.735556,...,11.582568,55.798500,0.005862,26.379541,13.118188,0.0,2.040178,6.677308,4518.0,-2.7800
3,4,DEER,0.10,A,2004,2453184.5,Lightning,CA,38.559167,-119.913333,...,8.259057,76.284248,0.004474,20.343042,7.937524,0.0,2.230986,0.000000,2384.0,-2.6175
4,5,STEVENOT,0.10,A,2004,2453184.5,Lightning,CA,38.559167,-119.933056,...,8.259057,76.284248,0.004474,20.343042,7.937524,0.0,2.230986,0.000000,2384.0,-2.6175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121323,300348363,ODESSA 2,0.01,A,2015,2457291.5,Missing/Undefined,CA,40.481637,-122.389375,...,8.107708,76.157257,0.006148,33.893488,16.492151,0.0,2.316175,40.920891,2216.0,-3.3700
121324,300348373,None,0.20,A,2015,2457300.5,Miscellaneous,CA,37.617619,-120.938570,...,8.788897,64.763962,0.006864,26.668207,13.650201,0.0,1.466999,646.360901,2263.0,-2.6900
121325,300348375,None,0.10,A,2015,2457144.5,Missing/Undefined,CA,37.617619,-120.938570,...,10.918760,63.418705,0.006537,32.813928,13.751520,0.0,3.065458,646.360901,2653.0,-3.1725
121326,300348377,None,2.00,B,2015,2457309.5,Missing/Undefined,CA,37.672235,-120.898356,...,9.216611,67.622978,0.007353,32.346643,17.716425,0.0,3.537143,1565.474243,2068.0,-2.7975


In [45]:
merged_df.to_csv("/content/drive/MyDrive/Spring 2025/ML and Climate/wildfire_project/ca_2000_mean_fire_earth_data.csv", index=False)